In [1]:
# -*- coding: utf-8 -*-
from get_response import PoliGPT
from typing import Any, Dict, List, Sequence, Mapping # Mapping es más general que Dict
from datasets import Dataset

# --- NECESITAS DEFINIR ESTAS RESPUESTAS DE REFERENCIA ---
# Estas son las respuestas "correctas" o "ideales" para cada pregunta.
# Tendrás que escribirlas tú mismo basándote en el conocimiento esperado.
# Ejemplo (DEBES RELLENARLAS CON LAS RESPUESTAS CORRECTAS REALES):
ground_truths: Dict[str, str] = {
    "¿Es necesario el conocimiento de Valenciano para acceder a una plaza en la Universidad?": "",
    "¿Cuáles son los requisitos para estudiar en la Universidad?": "",
    "¿Hay becas disponibles para estudiantes?": "",
    "¿Qué tipo de apoyo académico se ofrece a los estudiantes?": "",
    "¿Cómo se puede acceder a la información sobre las becas?": "",
}
# ---------------------------------------------------------


def build_eval_dataset(
    client: "PoliGPT",
    questions: Sequence[str],
    ground_truths: Mapping[str, str], # Añadido: Acepta las respuestas de referencia
    k_context: int = 3
) -> Dataset:
    """Construye un *Dataset* de `datasets` compatible con *Ragas*.

    Args:
        client: Instancia de :class:`PoliGPT`.
        questions: Lista de preguntas a evaluar.
        ground_truths: Diccionario mapeando preguntas a sus respuestas de referencia.
        k_context: Nº de fragmentos de contexto que se solicitarán a PoliGPT.

    Returns:
        Dataset con columnas 'question', 'contexts', 'answer' y 'reference'.
    """
    rows: Dict[str, List[Any]] = {
        "question": [],
        "contexts": [],
        "answer": [],
        "reference": [] # Añadida la columna para las respuestas de referencia
    }

    for q in questions:
        # Asegúrate de que hay una respuesta de referencia para esta pregunta
        if q not in ground_truths:
            print(f"Advertencia: No se encontró respuesta de referencia (ground truth) para la pregunta: '{q}'. Saltando...")
            continue

        res = client.query_poligpt(q, k_context=k_context)

        # Salta en caso de error en la consulta a PoliGPT
        if "error" in res:
            print(f"Pregunta ignorada por falta de contexto o error en PoliGPT: '{q}'")
            continue

        rows["question"].append(q)
        rows["answer"].append(res["response"])
        rows["contexts"].append([c["content"] for c in res["contexts"]])
        rows["reference"].append(ground_truths[q]) # Añade la respuesta de referencia correspondiente

    # Verifica que todas las listas tengan la misma longitud antes de crear el Dataset
    list_lengths = {key: len(value) for key, value in rows.items()}
    if len(set(list_lengths.values())) > 1:
        print("Error: Las listas de columnas tienen diferentes longitudes:", list_lengths)
        # Decide cómo manejar esto: ¿lanzar un error, devolver un dataset vacío, etc.?
        # Por ahora, simplemente imprimimos y devolvemos un dataset vacío o parcial
        # Podrías intentar filtrar las filas incompletas si es apropiado
        non_empty_rows = {k: v for k, v in rows.items() if v}
        if not non_empty_rows or len(set(len(lst) for lst in non_empty_rows.values())) > 1 :
             return Dataset.from_dict({"question":[], "contexts":[], "answer":[], "reference":[]}) # O lanzar excepción
        # Intentar crear con las columnas que sí tienen datos (si todas tienen la misma longitud > 0)
        min_len = min(len(v) for v in non_empty_rows.values())
        rows_truncated = {k: v[:min_len] for k, v in non_empty_rows.items()}
        print("Advertencia: Creando dataset con datos truncados debido a longitudes desiguales.")
        return Dataset.from_dict(rows_truncated)


    # Si no hay datos válidos después de filtrar, devuelve un dataset vacío
    if not rows["question"]:
         print("Advertencia: No se generaron filas válidas para el dataset de evaluación.")
         return Dataset.from_dict({"question":[], "contexts":[], "answer":[], "reference":[]})


    return Dataset.from_dict(rows)

In [2]:
# --- Script Principal ---

preguntas = [
    #"¿Es necesario el conocimiento de Valenciano para acceder a una plaza en la Universidad?",
    "¿Cuáles son los requisitos para estudiar en la Universidad?",
    #"¿Qué carreras se ofrecen en la Universidad?",
    #"¿Hay becas disponibles para estudiantes?",
    #"¿Qué tipo de apoyo académico se ofrece a los estudiantes?",
    #"¿Cómo se puede acceder a la información sobre las becas?",
]

# Cargar el cliente PoliGPT
# Asegúrate de que la ruta al índice FAISS sea correcta
try:
    poligpt = PoliGPT(faiss_index_dir='../01_data/project_faiss')
except Exception as e:
    print(f"Error al cargar PoliGPT: {e}")
    # Salir o manejar el error adecuadamente
    exit()

# Construir el dataset de evaluación, pasando las respuestas de referencia
eval_ds = build_eval_dataset(poligpt, preguntas, ground_truths, k_context=3)

# Verificar si el dataset se construyó correctamente
print("Dataset construido:")
print(eval_ds)

for i in range(len(eval_ds)):
    print(f"Pregunta: {eval_ds['question'][i]}")
    print(f"Respuesta: {eval_ds['answer'][i]}")
    print(f"Contextos: {eval_ds['contexts'][i]}")
    print(f"Referencia: {eval_ds['reference'][i]}")
    print("-" * 40)

FAISS inicializado - Vectores: 79044 | Dimensión: 768
Dataset construido:
Dataset({
    features: ['question', 'contexts', 'answer', 'reference'],
    num_rows: 1
})
Pregunta: ¿Cuáles son los requisitos para estudiar en la Universidad?
Respuesta: Los requisitos para estudiar en la Universidad son: 
a) Estar matriculado y tener consideración de alumno de la Universitat Politècnica de València.
b) En el caso de prácticas externas curriculares para titulaciones de grado, máster y títulos propios, estar matriculado en los créditos de prácticas y cumplir los requisitos especificados en el plan de estudios y/o por la estructura responsable del título.
c) En el caso de prácticas externas curriculares para titulaciones de grado. 

No se proporcionan más detalles sobre otros posibles requisitos, como puede ser el caso de doctorados u otras modalidades educativas.
Contextos: ['estudiants han de complir els requisits següents:\na) Estar matriculat i tenir consideració d’alumne\nde la Universitat 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# Cargar el modelo y el tokenizador
model_id = "gpt2"  # Puedes elegir un modelo más pequeño si es necesario
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Crear el pipeline de generación de texto
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    device=0  # Asegúrate de que la GPU esté disponible
)

# Envolver el pipeline con LangChain
llm = HuggingFacePipeline(pipeline=text_generator)


Device set to use cuda:0
/tmp/ipykernel_42438/578261077.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generator)


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")


/tmp/ipykernel_42438/1552202866.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")


In [3]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from datasets import Dataset

# Supongamos que tienes un dataset estructurado con las siguientes columnas:
# 'question', 'answer', 'contexts', 'ground_truth'
data = {
    "question": ["¿Cuál es la capital de Francia?"],
    "answer": ["La capital de Francia es París."],
    "contexts": [["París es la capital y ciudad más poblada de Francia."]],
    "ground_truth": ["París"]
}
dataset = Dataset.from_dict(data)

# Evaluar el sistema RAG
results = evaluate(
    dataset=dataset,
    llm=llm,
    embeddings=embedding_model,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)

# Mostrar los resultados
for metric_name, score in results.items():
    print(f"{metric_name}: {score:.2f}")


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1056 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [386,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLargeIndex: block: [386,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1553: indexSelectLarg

KeyboardInterrupt: 